# Example selectors

https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/

In [1]:
# creating simple translation examples
examples = [
    {"input": "hi", "output": "ciao"},
    {"input": "bye", "output": "arrivaderci"},
    {"input": "soccer", "output": "calcio"},
]

### Create a custom example selector

In [2]:
# create a custom example selector which picks examples based on the length of the input
from langchain_core.example_selectors.base import BaseExampleSelector


class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        # This assumes knowledge that part of the input will be a 'text' key
        new_word = input_variables["input"]
        new_word_length = len(new_word)

        # Initialize variables to store the best match and its length difference
        best_match = None
        smallest_diff = float("inf")

        # Iterate through each example
        for example in self.examples:
            # Calculate the length difference with the first word of the example
            current_diff = abs(len(example["input"]) - new_word_length)

            # Update the best match if the current one is closer in length
            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match]

In [3]:
example_selector = CustomExampleSelector(examples)

In [4]:
example_selector.select_examples({"input": "okay"})

[{'input': 'bye', 'output': 'arrivaderci'}]

In [5]:
example_selector.add_example({"input": "hand", "output": "mano"})

In [6]:
example_selector.select_examples({"input": "okay"})

[{'input': 'hand', 'output': 'mano'}]

### Use in a prompt

In [7]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# this will be used to format output from the example selector
example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

In [14]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="Translate the following words from English to Italain:",
    input_variables=["input"],
)

print(prompt.format(input="word"))

Translate the following words from English to Italain:

Input: hand -> Output: mano

Input: word -> Output:
